**1. MODULE ENVIRONNMENT**
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation énergétique des Data Center
# Import des libraries/
# Importing the libraries

In [ ]:
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation 2nergétique des Data Center
# Import des libraries/# Importing the libraries
import numpy as np #Pour la creation des matricesm vecteurs et fonction maths

#Creation de l'environement dans une classe/# BUILDING THE ENVIRONMENT IN A CLASS
class Environnment(object):
    # Introduction et initialisation de tous les parametres et variables de l'environnement
    # INTRODUCING AND INITIALIZING ALL THE PARAMETERS AND VARIABLES OF THE ENVIRONMENT
    def __init__(self, optimal_temperature = (18.0,24.0), initial_month = 0,initial_number_users = 10, initial_rate_data=60) :
        self.monthly_atmospheric_temperatures = [1.0, 5.0, 7.0, 10.0, 11.0, 20.0,23.0, 24.0,22.0, 10.0, 5.0, 1.0]#Les temperatures atmospheriques mois par mois
        self.initial_month = initial_month
        self.atmospheric_temperature =self.monthly_atmospheric_temperatures[initial_month]#Temperature atmosherique actuelle
        self.optimal_temperature = optimal_temperature
        self.min_temperature = -20
        self.max_temperature = 80
        self.min_number_users = 10
        self.max_number_users = 100
        self.max_update_users = 5
        self.min_rate_data = 20
        self.max_rate_data = 300
        self.max_update_data = 10
        self.initial_number_users = initial_number_users
        self.current_number_users = initial_number_users
        self.initial_rate_data = initial_rate_data
        self.current_rate_data = initial_rate_data
        self.intrinsic_temperature = self.atmospheric_temperature +1.25 * self.current_number_users+1.25 * self.current_rate_data
        self.temperature_ai = self.intrinsic_temperature
        self.temperature_noai = (self.optimal_temperature[0]+ self.optimal_temperature[1]) / 2.0
        self.total_energy_ai = 0.0
        self.total_energy_noai = 0.0
        self.reward = 0.0
        self.game_over = 0#indique si on a terminer ou non
        self.train = 1 # defini si on est en mode entrainement ou test

    # Creation d'une methode pour la mise a jour de l'environnement a chaque fois que l'IA ait mener une action## MAKING A METHOD THAT UPDATES THE ENVIRONMENT RIGHT AFTER THE AI PLAYS AN ACTION
    def update_env(self, direction, energy_ai, month):
        # methode de mise a jour de l'environnement
        # GETTING THE REWARD#OBTENIR LA RECOMPENSE
        # Computing the energy spent by the server’s cooling system when there is no AI# Calcul de l'energie interne sans IA
        energy_noai = 0
        if (self.temperature_noai < self.optimal_temperature[0]):
            energy_noai = self.optimal_temperature[0] - self.temperature_noai
            self.temperature_noai = self.optimal_temperature[0]#mise a jour de la nouvelle temperature
        elif (self.temperature_noai > self.optimal_temperature[1]):
            energy_noai = self.temperature_noai - self.optimal_temperature[1]
            self.temperature_noai = self.optimal_temperature[1]
        # Computing the Reward # Calcul de la recompense
        self.reward = energy_noai - energy_ai
        # Scaling the Reward# Pour normliser la recompense obtenue
        self.reward = 1e-3 * self.reward
        # GETTING THE NEXT STATE# CHANGER L'ENVIRONEMENT(au temps t+1)
        # Updating the atmospheric temperature# Mise a jour de la temperature atmospherique
        self.atmospheric_temperature = self.monthly_atmospheric_temperatures[month]
        # Updating the number of users# Mise a jour du nombre d'utilisateurs
        self.current_number_users += np.random.randint(-self.max_update_users, self.max_update_users)
        if (self.current_number_users > self.max_number_users):
            self.current_number_users = self.max_number_users
        elif (self.current_number_users < self.min_number_users):
            self.current_number_users = self.min_number_users
        # Updating the rate of data# Mise a jour du taux de transmission de donnees
        self.current_rate_data += np.random.randint(-self.max_update_data,self.max_update_data)
        if (self.current_rate_data > self.max_rate_data):
            self.current_rate_data = self.max_rate_data
        elif (self.current_rate_data < self.min_rate_data):
            self.current_rate_data = self.min_rate_data
        # Computing the Delta of Intrinsic Temperature# Calcul de levolution de la temperature intrinsec du serveur
        past_intrinsic_temperature = self.intrinsic_temperature
        self.intrinsic_temperature = self.atmospheric_temperature + 1.25 * self.current_number_users + 1.25 * self.current_rate_data
        delta_intrinsic_temperature = self.intrinsic_temperature - past_intrinsic_temperature
        # Computing the Delta of Temperature caused by the AI#
        if (direction == -1):
            delta_temperature_ai = -energy_ai
        elif (direction == 1):
            delta_temperature_ai = energy_ai
        # Updating the new Server’s Temperature when there is the AI
        self.temperature_ai += delta_intrinsic_temperature + delta_temperature_ai
        # Updating the new Server’s Temperature when there is no AI
        self.temperature_noai += delta_intrinsic_temperature
        # GETTING GAME OVER# DETERMINER SI ON A UN GAME OVER OU NON C'EST-A-DIRE ARRETER L'ENTRAINEMENT OU NON
        if (self.temperature_ai < self.min_temperature):
            if (self.train == 1):
                self.game_over = 1
            else:
                self.total_energy_ai += self.optimal_temperature[0] - self.temperature_ai
                self.temperature_ai = self.optimal_temperature[0]
        elif (self.temperature_ai > self.max_temperature):
            if (self.train == 1):
                self.game_over = 1
            else:
                self.total_energy_ai += self.temperature_ai - self.optimal_temperature[1]
                self.temperature_ai = self.optimal_temperature[1]

        # UPDATING THE SCORES# MISE A JOUR DE SCORE (DANS NOTRE CAS ICI C'EST L'ENERGIE)

        # Updating the Total Energy spent by the AI
        self.total_energy_ai += energy_ai
        #Updating the Total Energy spent by the alternative system when there is no AI
        self.total_energy_noai += energy_noai
        #PREPARATION DES IMPUT POUR LE RESEAU DE NEURONE
        # SCALING THE NEXT STATE# CHANGER L'ECHELLE OU ENCORE LA NORMALISATION
        scaled_temperature_ai = (self.temperature_ai - self.min_temperature) / (self.max_temperature - self.min_temperature)
        scaled_number_users = (self.current_number_users - self.min_number_users)/ (self.max_number_users - self.min_number_users)
        scaled_rate_data = (self.current_rate_data - self.min_rate_data) / (self.max_rate_data - self.min_rate_data)
        next_state = np.matrix([scaled_temperature_ai, scaled_number_users, scaled_rate_data])#Matrice contenant les IMPUT ppour le reseau de neurone

        # RETURNING THE NEXT STATE, THE REWARD, AND GAME OVER# RETOURNER L'ETAT SUIVANT, LA RECOMPENSE ET LE GAME OVER
        return next_state, self.reward, self.game_over


        # MAKING A METHOD THAT RESETS THE ENVIRONMENT# METHODE DE REINITIALISATION DE L'ENVIRONNEMENT
    def reset(self, new_month):
        self.atmospheric_temperature = self.monthly_atmospheric_temperatures[new_month]
        self.initial_month = new_month
        self.current_number_users = self.initial_number_users
        self.current_rate_data = self.initial_rate_data
        self.intrinsic_temperature = self.atmospheric_temperature + 1.25 * self.current_number_users + 1.25 * self.current_rate_data
        self.temperature_ai = self.intrinsic_temperature
        self.temperature_noai = (self.optimal_temperature[0] + self.optimal_temperature[1]) / 2.0
        self.total_energy_ai = 0.0
        self.total_energy_noai = 0.0
        self.reward = 0.0
        self.game_over = 0
        self.train = 1
        # MAKING A METHOD THAT GIVES US AT ANY TIME THE STATE, THE REWARD AND GAMEOVE# Pour savoir l'etat actuel de l'environnement a chaque minute
        # et la derniere recompense
    def observe(self):
        scaled_temperature_ai = (self.temperature_ai - self.min_temperature)/ (self.max_temperature - self.min_temperature)
        scaled_number_users = (self.current_number_users - self.min_number_users)/ (self.max_number_users - self.min_number_users)
        scaled_rate_data = (self.current_rate_data - self.min_rate_data)/ (self.max_rate_data - self.min_rate_data)
        current_state = np.matrix([scaled_temperature_ai,scaled_number_users,scaled_rate_data])
        return current_state, self.reward, self.game_over
        # Computing the energy spent by the server’s cooling system when there is no AI

    # Creation d'une methode qui nous donnera a chaque temps t l'etat actuel dans lequel on se trouve,la derniere recompense obtenue et si on a terminer



**2. MODULE BRAIN**
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation 2nergétique des Data Center
# Building the Brain
# Developpement du cerveau

In [ ]:
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation 2nergétique des Data Center
# Building the Brain# Developpement du cerveau

# Importing the libraries# Import des librairies necessaires
#rom tensorflow.keras.layers import Input, Dense, Dropout
#from tensorflow.keras.models import Model
#from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam # un optimizeur
#from keras.src import tree

# BUILDING THE BRAIN# CREATION DU CERVEAU

class Brain(object):
    def __init__(self, learning_rate=0.001, number_actions=5):
        self.learning_rate = learning_rate
        # BUILDIND THE INPUT LAYER COMPOSED OF THE INPUT STATE# CREATION DE LA  COUCHE D'ENTREE
        states = Input(shape = (3,))
        # BUILDING THE FULLY CONNECTED HIDDEN LAYERS# LES COUCHES CACHEES
        x = Dense(units = 64, activation = 'sigmoid')(states)
        y = Dense(units = 32, activation = 'sigmoid')(x)
        # BUILDING THE OUTPUT LAYER, FULLY CONNECTED TO THE LAST HIDDEN LAYER# CREATION DE LA  COUCHE DE SORTIE
        q_values = Dense(units = number_actions, activation = 'softmax')(y)# Sofmax est beaucoup plus utiliser pour la classification car elle permet d'obtenir les probabilités
        # ASSEMBLING THE FULL ARCHITECTURE INSIDE A MODEL OBJECT#
        self.model = Model(inputs = states, outputs = q_values)# CREATION DU MODELE
        # COMPILING THE MODEL WITH A MEAN-SQUARED ERROR LOSS AND A CHOSEN OPTIMIZER# COMPILATION DU MODELE
        self.model.compile(loss = 'mse', optimizer = Adam(learning_rate = learning_rate)) # mse pour les problemes de regressions



**3. MODULE DQN**
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation 2nergétique des Data Center
# Implementing Deep Q-Learning with Experience Replay
# Implementtion de l'algo deep q learning

In [ ]:
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation 2nergétique des Data Center
# Implementing Deep Q-Learning with Experience Replay# Implementtion de l'algo deep q learning

# Importing the libraries# Import des libraries
import numpy as np

class DQN(object):
    #INTRODUCING AND INITIALIZING ALL THE PARAMETERS AND VARIABLES OF THE DQN# INITIALISATION DE LA CLASSE
    def __init__(self, max_memory=100, discount=0.9):
        #self.memory = list()
        self.max_memory = max_memory
        self.discount = discount
        self.memory = list()
    # MAKING A METHOD THAT BUILDS THE MEMORY IN EXPERIENCE REPLAY# METHODE POUR LE PRECESSUS DE REMPLISSAGE DE L'EXPERIENCE REPLAY
    def remember(self, transition, game_over):
        self.memory.append([transition, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]#on supprime le tout premier element de la memoire
    # MAKING A METHOD THAT BUILDS TWO BATCHES OF INPUTS AND TARGE# CREATION OU OBTENTION DE DEUX BACHES(ENTRE ET LA CIBLE)
    def get_batch(self, model, batch_size = 10):
        len_memory = len(self.memory)
        num_inputs = self.memory[0][0][0].shape[1]
        num_outputs = model.output_shape[-1]
        #inputs = np.zeros((batch_size, num_inputs))
        #targets = np.zeros((batch_size, num_outputs))
        inputs = np.zeros((min(len_memory, batch_size), num_inputs))
        targets = np.zeros((min(len_memory, batch_size), num_outputs))
        transitions = np.random.randint(0, len_memory,size = min(len_memory, batch_size))
        for i, idx in enumerate(transitions):
            current_state, action, reward, next_state = self.memory[idx][0]
            game_over = self.memory[idx][1]
            inputs[i] = current_state
            #targets[i, action] = model.predict(current_state)[0]
            targets[i, action] = reward + self.discount * np.max(model.predict(next_state)[0])# equation de Belman
            #Q_sa = np.max(model.predict(next_state)[0])
            if game_over==1:
                targets[i, action] = reward
            else:
                targets[i, action] = reward + self.discount * np.max(model.predict(next_state)[0])
        return inputs, targets




**4. MODULE TRAINING**
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation 2nergétique des Data Center
# Training the AI
# Installing Keras
 # conda install -c conda-forge keras
# Importing the libraries and the other python files

In [ ]:
#Mise en œuvre d’un algorithme de Deep Q-Learning intégrant
#le modèle de régression linéaire multiple pour l’optimisation 2nergétique des Data Center
# Training the AI
# Installing Keras
 # conda install -c conda-forge keras
# Importing the libraries and the other python files

import os # interaction avec le systeme d'exploitation
import numpy as np
import random as rn
#import environnment
#import brain
#import dqn

# Setting seeds for reproducibility
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)# avoir les memes resultats lorsqu'on genere les nombres aleatoires
# SETTING THE PARAMETERS # LES PARAMETRES GLOBEAUX
epsilon = 0.3 # le parametre d'exploration (30%). Quand on entraine une IA, il fo tjrs trouver le juste milieu entre l'exploration et l'exploitation
number_actions = 5
direction_boundary = (number_actions - 1) / 2 # la frontiere de direction(pour trouver le juste milieu)
number_epochs = 100 # nombre d'epoque ( le nombre de fois qu'on fera lentrainement sur un nouveau serveur)
max_memory = 3000
batch_size = 512
temperature_step = 1.5
#BUILDING THE ENVIRONMENT BY SIMPLY CREATING AN OBJECT OF THE ENVIRONMENT CLASS# CREATION DE L'ENVIRONNEMENT
env = Environnment(optimal_temperature=(18.0, 24.0), initial_month = 0, initial_number_users = 20, initial_rate_data = 30)
# BUILDING THE BRAIN BY SIMPLY CREATING AN OBJECT OF THE BRAIN CLASS# CREATION DU CERVEAU
brain = Brain(learning_rate = 0.00001, number_actions = number_actions)
# BUILDING THE DQN MODEL BY SIMPLY CREATING AN OBJECT OF THE DQN CLASS# CONSTRUCTION DU DEEP Q-LEARNING
dqn = DQN(max_memory = max_memory, discount = 0.9)
# CHOOSING THE MODE# CHOISIR SI ON EST EN PHASE D'ENTRAINEMENT OU DE TEST
train = True

# TRAINING THE AI# ENTRAINEMENT DE L'IA
env.train = train
model = brain.model
early_stopping = True
patience = 10
best_total_reward = -np.inf
patience_count = 0
if (env.train):
    # STARTING THE LOOP OVER ALL THE EPOCHS (1 Epoch = 5 Months)
    for epoch in range(1, number_epochs):
    #INITIALIAZING ALL THE VARIABLES OF BOTH THE ENVIRONMENT AND THE TRAINING LOOP
        total_reward = 0
        loss = 0. # la fonction cout
        new_month = np.random.randint(0, 12)
        env.reset(new_month = new_month)#remettre a zero lenvironnement
        game_over = False
        current_state, _, _ = env.observe()# retourner 3 objets(letat initial, 0,0)
        timestep = 0
        # STARTING THE LOOP OVER ALL THE TIMESTEPS (1 Timestep = 1 Minute) IN ONE EPOCH
        while ( timestep <= 5 * 30 * 24 * 60 and (not game_over)):
            #tant que le timestep est plut petit
            # PLAYING THE NEXT ACTION BY EXPLORATION ACTION PRISE PAR EXPLORATION
            if np.random.rand() <= epsilon:
                action = np.random.randint(0, number_actions)
                if (action < direction_boundary):
                    direction = -1
                else:
                    direction = 1
                energy_ai = abs(action - direction_boundary) * temperature_step
            else:
                q_values = model.predict(current_state)[0]
                action = np.argmax(q_values)
                if (action < direction_boundary):
                    direction = -1
                else:
                    direction = 1
                energy_ai = abs(action - direction_boundary) * temperature_step

            # UPDATING THE ENVIRONMENT AND REACHING THE NEXT STATE
            month = new_month + int(timestep / (30*24*60))
            if month >= 12:
                month -= 12
            next_state, reward, game_over = env.update_env(direction, energy_ai, month)
            total_reward += reward
            # STORING THIS NEW TRANSITION INTO THE MEMORY
            transition = [current_state, action, reward, next_state]
            dqn.remember(transition, game_over)
            # GATHERING IN TWO SEPARATE BATCHES THE INPUTS AND THE TARGETS
            inputs, targets = dqn.get_batch(model, batch_size = batch_size)
            # COMPUTING THE LOSS OVER THE TWO WHOLE BATCHES OF INPUTS AND TARGETS
            loss += model.train_on_batch(inputs, targets)
            timestep += 1
            current_state = next_state
        # PRINTING THE TRAINING RESULTS FOR EACH EPOCH
        print("\n")
        print("Epoch: {:03d}/{:03d}".format(epoch, number_epochs))
        print("Total energy spent with an AI:{:.0f}".format(env.total_energy_ai))
        print("Total energy spent with no AI:{:.0f}".format(env.total_energy_noai))
        # SAVING THE MODEL
        model.save("model.h5")
        model.save("my_model.keras")
